In [1]:
import pandas as pd
import numpy as np

import datetime
from dateutil.parser import parse
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
ohe = OneHotEncoder()
le = LabelEncoder()

In [6]:
raw_path = '../data/1.raw_csv/'
path = '../data/2.Preprocessed/'

## Accounts

In [7]:
accounts = pd.read_csv(raw_path + 'accounts.csv')

#### accounts: 최근 접속 일자 변환

In [8]:
accounts['last_login_dts'] = pd.to_datetime(accounts['last_login_dts']*1000000)
accounts.dropna(inplace=True)

#### accounts: gender 변환

In [9]:
accounts.gender.unique()

array(['M', 'F', '-', '0'], dtype=object)

In [10]:
accounts['gender'].replace(to_replace = '-', value = np.nan, inplace = True)
accounts['gender'].replace(to_replace = '0', value = np.nan, inplace = True)
accounts.dropna(inplace=True)
accounts = pd.get_dummies(accounts, columns=['gender'])

#### accounts: age 결측처리

In [11]:
accounts.age.unique()

array([ 43.,  40.,  57.,  45.,  52.,  49.,  48.,  46.,  47.,  42.,  41.,
        37.,  35.,  66.,  56.,  44.,  50.,  51.,  34.,  58.,  72.,  59.,
        53.,  36.,  74.,  54.,  55.,  61.,  39.,  63.,  38.,  32.,  69.,
        65.,  64.,  30.,  60.,  62.,  75.,  31.,  33.,  29.,  67.,  17.,
        73.,  28.,  26.,  27.,  24.,  77.,  68.,  70.,  25.,  79.,  76.,
        23.,  85.,  21.,  18.,  20.,  22.,  12.,  19.,  14.,  71.,  11.,
        15., 100.,  16.,  81.,  13.,  -1.,  86.,  10.,  80.,   8.,   9.,
        83.,  78.,  94.,  82.,  90.,  95.,  88.,   4.,  87.,  89.,  84.,
        91.,  93.,  92.,  96.,  98.,  99.,   6.,   5.,  97.,   7.,   2.,
         3.])

In [12]:
accounts['age'].replace(to_replace = -1, value = np.nan, inplace = True) # 결측값 = -1
accounts['age'] = accounts['age'].apply(lambda x : np.nan if x <=7 else x) # 7세 이하는 결측처리

In [13]:
accounts.dropna(inplace=True)
accounts['age'].isnull().sum()

0

#### accounts: age 이산화

In [14]:
accounts['age'] = pd.cut(accounts['age'], [8, 12, 19, 30, 42, 51, 64, np.inf],
                        labels=['0', '1', '2', '3', '4', '5', '6'], right=False)

In [60]:
accounts = pd.get_dummies(accounts, columns=['age'])

#### accounts: 주소

In [15]:
accounts['address'][0].split(' ')[0]

'경기도'

In [16]:
# 수도권만 1, 나머지 0으로 설정
accounts['address_is'] = accounts['address'].apply(lambda x: 0 if '경기' in x.split(' ')[0] else 1  if '인천' in x.split(' ')[0] else 1 if '서울' in x.split(' ')[0] else 1)

In [17]:
accounts.dropna(inplace=True)
accounts['address_is'].isnull().sum()

0

In [58]:
accounts = pd.get_dummies(accounts, columns=['address_is'])

In [63]:
gen_col = [col for col in accounts.columns if col.startswith('gender')]
add_col = [col for col in accounts.columns if col.startswith('address_is')]
age_col = [col for col in accounts.columns if col.startswith('age')]

#### 저장

In [67]:
accounts = accounts[['account_id', 'last_login_dts'] + gen_col + add_col + age_col]
accounts.to_csv(path + 'accounts_preprocessed.csv',encoding='utf-8', index=None)

## Orders

### orders: java시간 변환

In [19]:
orders = pd.read_csv(raw_path + 'orders.csv')

In [22]:
def invert_javatime(javatime):
  seconds = javatime / 1000
  sub_seconds = (javatime % 1000.0) / 1000.0
  date = datetime.datetime.fromtimestamp(seconds + sub_seconds)
  return date

In [23]:
java_timestamp = 1589424016190
invert_javatime(java_timestamp)

datetime.datetime(2020, 5, 14, 11, 40, 16, 380000)

In [24]:
orders['created_at'] = orders['created_at'].apply(lambda x : invert_javatime(x))
orders

,order_id,account_id,product_id,price,created_at
0,218519157,100777,84936431,10800.0,2020-03-27 10:41:36.286
1,221364615,100777,23116461,6300.0,2020-05-03 18:31:28.266
2,218519157,100777,86641767,14850.0,2020-03-27 10:41:36.286
3,223489288,100777,6315392,10800.0,2020-06-03 14:21:50.474
4,218519157,100777,85010480,43200.0,2020-03-27 10:41:36.286
...,...,...,...,...,...
8382509,223843200,16987547,3799797,9000.0,2020-06-08 23:50:52.854
8382510,223850682,16988206,73776838,18000.0,2020-06-09 07:09:55.166
8382511,223871729,16989204,90496918,66600.0,2020-06-09 13:48:23.986
8382512,223887913,16990132,7217755,12600.0,2020-06-09 18:12:26.054


### 저장

In [25]:
orders.to_csv(path+'orders_preprocessed.csv',encoding='utf-8')

## Products

In [26]:
products = pd.read_csv(raw_path+'products.csv',dtype={'published_at':str})

In [27]:
products = products[products['published_at'].notna()]

### products: 출판일 전처리(date형식)

In [28]:
products['published_at'] = products['published_at'].apply(lambda x : x[0:8] if type(x) == 'str' else x)
products['published_at'] = products['published_at'].apply(lambda x : x.strip() if type(x) == 'str' else x)
products['published_at'] = products['published_at'].apply(lambda x : '20991231' if len(x) <8 else x)
products['published_at'] = products['published_at'].astype(str)
products['published_at'] = products['published_at'].apply(lambda x : '20991231' if int(x[4:6]) >= 13 else x )
products['published_at'] = products['published_at'].apply(lambda x : '20991231' if int(x) <= 19000000 else x)

In [29]:
products['published_at'] = products['published_at'].apply(lambda x : parse(x) )

In [30]:
products = products[products['published_at'].isin(pd.date_range('1800-01-01', '2020-12-31'))]

### products: 출판일 이산화

In [31]:
default_date = '2020-05-01'
def date_range(days=None, default_date='2020-05-01'):
    default_date = pd.to_datetime(default_date)
    delta = datetime.timedelta(days=days)
    before_date = default_date - delta
    after_date = default_date + delta
    return pd.date_range(before_date, after_date, periods=2)

In [32]:
products['pub'] = 4
products.loc[products[products['published_at'].isin(date_range(days=1095))].index,'pub'] = 3
products.loc[products[products['published_at'].isin(date_range(days=365))].index,'pub'] = 2
products.loc[products[products['published_at'].isin(date_range(days=180))].index,'pub'] = 1
products.loc[products[products['published_at'].isin(date_range(days=90))].index,'pub'] = 0

In [33]:
products = pd.get_dummies(products, columns=['pub'])

### products: 카테고리ID 전처리

In [34]:
products['category_id'] = products['category_id'].astype(str)

In [35]:
products['category_id'] = products['category_id'].apply(lambda x: x[2:4] if len(x) > 3 else (x+((4-len(x))*'0'))[2:4])

In [36]:
products = products[products['category_id'] != '00']
products = pd.get_dummies(products, columns=['category_id'])

### products: 가격대 이상치 처리 및 이산화

In [37]:
condition1 = products['shop_price']>3500000.0 # 3500000 만원 이상은 고서적, 일반인들이 구매하지 않을 종류의 책들, 300000~3500000 가격의 책들은 대부분 전집에 해당
condition2 = products['shop_price']<1000.0 # 1000 미만은 사은품 등의 다른 제품들이 혼재

In [38]:
products = products[~condition1 & ~condition2]
products = products[products['shop_price'] != 0.0]

In [39]:
products['shop_price'] = pd.qcut(products['shop_price'], 4, labels=['0', '1', '2', '3'])

In [40]:
products = pd.get_dummies(products, columns=['shop_price'])

In [41]:
products.to_csv(path+'products_preprocessed.csv',encoding='utf-8', index=None)

## click

In [42]:
click = pd.read_csv(raw_path+'log.csv')

In [43]:
print(click.shape)
click.head()

(30113602, 6)


,Unnamed: 0,request_date_time,account_id,device_type,product_id,before_product_id
0,0,2020-03-14 10:39:45,101496,M,88429694,0
1,1,2020-03-24 15:33:35,101496,M,64694842,0
2,2,2019-12-31 09:54:33,101496,M,81503945,0
3,3,2019-12-31 09:52:14,101496,M,84434768,0
4,4,2019-12-31 09:52:25,101496,M,81503945,0


### click: device_type 원핫인코딩

In [44]:
click = pd.get_dummies(click, columns=['device_type'])

In [45]:
click.isnull().sum()

Unnamed: 0           0
request_date_time    0
account_id           0
product_id           0
before_product_id    0
device_type_M        0
device_type_P        0
dtype: int64

In [46]:
click.to_csv(path+'click.csv',encoding='utf-8', index=None)

## click & orders

### click & orders: orders 중 click없이 구매한 데이터(외부 사이트 이용) 삭제하기
* click 데이터 크기가 방대하여 절반만 사용한 관계로 더 적어짐

In [47]:
orders = pd.read_csv(path+'orders_preprocessed.csv',encoding='utf-8')

In [48]:
order_check_account = click['account_id'].unique().tolist()
order_check_product = click['product_id'].unique().tolist()

In [49]:
orders.shape

(8382514, 6)

In [50]:
order_check_1 = orders[orders['product_id'].isin(order_check_product)]

In [51]:
order_check_1.shape

(8344774, 6)

In [52]:
order_check_2 = order_check_1[order_check_1['account_id'].isin(order_check_account)]

In [53]:
order_check_2.shape

(4035057, 6)

In [54]:
order_check_2 = order_check_2.iloc[:,1:]

In [55]:
order_check_2.to_csv(path+'orders_only_in_clicks.csv',encoding='utf-8', index=None)